In [7]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
from numpy import character
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 3. TURN: 1 


你深吸一口气，鼓起勇气，走进了豪宅。

随着你的进入，一股阴冷的空气迎面扑来，让你不禁打了个寒颤。
豪宅内部昏暗而阴森，只有一些窗户透出微弱的光线。
你站在一个宽敞的大厅里，古老的家具布满灰尘，挂在墙上的肖像画似乎在审视着你。

你决定探索豪宅的更多区域。

你的左手边是一扇通往餐厅的门，门微开，里面似乎有动静。
你的右手边是一条通往书房的走廊，走廊尽头似乎有微弱的光线。

你要去哪里？

possible actions:
1. 进入餐厅。
2. 走向书房。
3. 在大厅继续调查。(可能会找到有用的线索)

I select option 3. TURN: 2 


你决定在大厅里深入探索，希望能找到一些关于这座豪宅的线索。

在大厅的一角，你发现了一张古老的桌子，桌子上堆满了尘土。你仔细地擦去灰尘，发现桌子上放着一本厚重的客户留言簿，看起来已经很久没有人翻看过了。

你靠近去翻阅这本留言簿，希望能从中找到一些豪宅的历史或者之前访客的经历。

skill: library_use
difficulty: easy

I am making a skill check using LIBRARY_USE against a difficulty of EASY.
And I rolled a 63 for a result of SUCCESS. TURN: 3 


成功翻阅留言簿后，你发现了一些有趣的信息。

留言簿中充满了过去访客的记录，许多人提到了豪宅的美丽与其背后隐藏的悲惨故事。一条条留言中，你注意到了一个反复出现的主题：豪宅的地下室。多个条目提到了地下室中不可言喻的恐怖，以及一种不明力量的存在。有几个留言提到，豪宅的主人曾经在地下室进行过一些禁忌的仪式。

你意识到，这座豪宅的秘密可能与地下室有关。你决定继续调查，希望能找到通往地下室的入口。

在大厅的另一边，你注意到了一扇看似通往地下室的门，以及楼梯向上的方向，可能通往豪宅的其他居住区域。

你要去哪里？

possible actions:
1. 尝试找到并打开通往地下室的门。
2. 上楼探索豪宅的居住区域。
3. 回头去探索之前未进入的餐厅或书房。

I select option 3. TURN: 4 


决定回头探索之前未进入的区域，你回到了大厅，站在那两扇未探索的门前。

你回想起进入豪宅时的情况，决定这次进入餐厅看看。你有一种感觉，这里可能隐藏着豪宅的某些秘密。

轻推开餐厅的门，你发现餐厅内部布置得相当豪华，长桌上铺着细密的桌布，上面堆满了灰尘。突然，你听到了一些动静，好像是某种动物的声音。你警惕地环顾四周，尝试找出声音的来源。

突然，一只看起来异常凶猛的大狗从房间的另一头向你冲了过来，它的眼中闪烁着不寻常的光芒。

你必须迅速做出反应，躲避它的攻击。

possible actions:
1. Approach the winner to see if you can take a look inside the storage with her.
2. Talk to the auctioneer about upcoming auctions.
3. Leave the auction and head to a nearby diner to rethink your strategy.

I select option 1. TURN: 3 


You approach the tall, slender woman who won the auction, introducing yourself with a polite nod. She seems intrigued by your interest and, after a brief conversation, agrees to let you take a quick look inside the storage with her. As the roller shutter opens fully, the dim light reveals an assortment of items: old books, furniture that has seen better days, and what appears to be a collection of old, strange-looking artifacts.

possible actions:
1. Examine the strange-looking artifacts more closely.
2. Offer to help the woman move some of the items to get a better look at what's in the back.
3. Ask the woman about her interest in this particular storage unit.

I select option 1. TURN: 4 


skill: spot_hidden
difficulty: medium

I am making a skill check using SPOT_HIDDEN against a difficulty of MEDIUM.
And I rolled a 77 for a result of FAILURE. TURN: 5 


As you lean in to examine the strange-looking artifacts more closely, your eyes struggle to make out the finer details in the dim light of the storage unit. The shapes and symbols on the artifacts blur together, making it impossible for you to discern their significance or value. Frustrated by your inability to uncover anything useful, you step back, feeling a pang of disappointment.

The woman, noticing your interest in the artifacts, offers a sympathetic smile before turning her attention back to the task of assessing her newly acquired treasures.

round summary: Kyle Jackson, after bidding unsuccessfully at the storage auction, managed to gain a brief look inside the winning storage unit. Despite his attempt to examine some intriguing artifacts, his effort was hampered by a failure to spot any hidden details, leaving him with no choice but to step back an

I am making a skill check using DODGE against a difficulty of MEDIUM.
And I rolled a 39 for a result of SUCCESS. TURN: 5 


你及时作出反应，巧妙地躲避了那只大狗的冲击。它在错失目标后，发出一声低沉的咆哮，然后转身逃离了餐厅，消失在了豪宅深处的阴影中。

喘息之余，你感到一丝庆幸，同时也意识到这座豪宅中隐藏着不寻常的力量，甚至可能是超自然的存在。

在确保安全之后，你决定继续探索餐厅，希望能找到一些有用的线索。餐厅内部除了那张长桌，还有一个大型的壁炉，看起来已经很久没有使用过了。壁炉的一侧，你注意到了一扇小门，似乎通往豪宅的另一个区域。

就在你打算深入探索时，你突然听到了来自豪宅地下室的可怕声音，像是什么东西在哭泣，又像是在呻吟。

此时，你意识到你的调查已经到了一个关键时刻。豪宅的秘密就隐藏在地下室中，而你必须找到进入地下室的方法。

summary: 在成功躲避了一只凶猛大狗的攻击后，你在餐厅中发现了一扇通往豪宅另一区域的小门，并听到了来自地下室的可怕声音，意识到必须找到进入地下室的方法。


前倾提要:
"""
- 在成功躲避了一只凶猛大狗的攻击后，你在餐厅中发现了一扇通往豪宅另一区域的小门，并听到了来自地下室的可怕声音，意识到必须找到进入地下室的方法。
"""


现在你被地下室的尖叫所吸引，来到了地下室的入口。
你要做什么？

possible actions:
1. 进入地下室。
2. 拍摄地下室入口的照片。
3. 试着在黑暗中寻找光源。
